# Read an image for visually challenged

## Table of Contents
* [Problem Statement](#section-1)
* [Importing libraries and reading data](#section-2)
* [Data Understanding](#section-3)
* [Pre-processing the captions](#section-4)
* [Pre-processing the images](#section-5)
* [Extracting the Feature vector](#section-6)
* [Model Building](#section-7)
    - [Data preperation](#subsect-1)
    - [Build the Encoder](#subsect-2)
    - [Build the Attention model](#subsect-3)
    - [Build the Decoder](#subsect-4)
* [Model training & optimization](#section-8)
* [Model Evaluation](#section-9)
* [Conclusions](#section-10)

<a id="section-1"></a>
# Problem Statement

## Introduction
    The World Health Organization (WHO) has reported that approximately 285 million people are visually impaired worldwide, and out of these 285 million, 39 million are completely blind. It gets extremely tough for them to carry out daily activities, one of which is reading. From reading a newspaper or a magazine to reading an important text message from your bank, it is tough for them to read the text written in it.

    A similar problem they also face is seeing and enjoying the beauty of pictures and images. Today, in the world of social media, millions of images are uploaded daily. Some of them are about your friends and family, while some of them are about nature and its beauty. Understanding what is present in that image is quite a challenge for certain people who are suffering from visual impairment or who are blind.

    In an initiative to help them experience the beauty of the images, Facebook had earlier launched a unique feature earlier that can help blind people operate the app on their mobile phones. The feature could explain the contents of an image that their friends have posted on Facebook. So, say, if someone posted a picture with their dog in the park, the application would speak out the contents and may describe it like, “This image may contain a dog standing with a man around the trees.”
    
    In this project, the intention is to build a model that is similar to the one developed by Facebook so that a visually challenged person knows the contents of an image in the form of audio. 
    Flickr8K dataset is used in the project, the dataset is available in Kaggle.
    
## Approach
    Firstly the information in an image is converted into a text description, then using a text to speech API, the audio output is extracted from the text description.
    
    This problem statement is an application of both deep learning and natural language processing. The features of an image will be extracted by a CNN-based encoder and this will be decoded by an RNN model. To generate a more meaningful and descriptive caption
    The main important portion is to generate the text description. A deep learning model using a CNN-RNN model with an attention mechanism is employed to generate the caption 

<a id="section-2"></a>
# Importing libraries and reading data

#### Mandatory codeblock to facilitate running on Kaggle

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
'''
gTTS - (Google Text-to-Speech)is a Python library and CLI tool to interface with Google Translate text-to-speech API. 

playsound - The playsound module is a cross platform module that can play audio files. 
'''
!pip install gtts
!pip install playsound
!pip install ipython

In [3]:
#Import all the required libraries

import os                   # Pathname manipulation
import glob                 # For pattern matching of pathnames
import time                 # Provides functions related to time values
import random               # To generate random numbers
import re                   # Regular expressions
from tqdm import tqdm       # Used for creating Progress Meters or Progress Bars ('taqaddum' in arabic for progress)
# Module provides additional data structures for collections of data like defaultdict, ordereddict, counter, Deque, Namedtuple, Chainmap
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import seaborn as sns

# PIL - Python Imaging Library, adds many image processing features
from PIL import Image
from skimage import io

from sklearn.model_selection import train_test_split 

#model building 
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.translate.bleu_score import sentence_bleu
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop 
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras import Input, layers
from keras import optimizers

from gtts import gTTS
from playsound import playsound
from IPython import display

<a id="section-3"></a>
# Data Understanding
1.Importing the dataset and reading image & captions into two seperate variables

2.Visualising both the images & text present in the dataset

3.Creating a dataframe which summarizes the image, path & captions as a dataframe

4.Creating a list which contains all the captions & path

5.Visualising the top 30 occuring words in the captions


In [4]:
#Importing the dataset and reading the image into a seperate variable

images_folder_path = '/kaggle/input/flickr8k/Images'  
# alternative code> images_folder_path = os.path.join('/','kaggle','input','flickr8k','Images')

captions_file_path = os.path.join('/','kaggle','input','flickr8k','captions.txt')
# alternative code> captions_file_path = '/kaggle/input/flickr8k/captions.txt'

working_folder_path = os.path.join('/','kaggle','working')

# Creating a list which contains the path to the images
all_imgs_path_list = glob.glob(images_folder_path + '/*.jpg',recursive=True) 
print("The total images present in the dataset: ", len(all_imgs_path_list))

In [5]:
#Visualising both the images & text present in the dataset

# Visualizing a random image
# Random image index
random_img_index = np.random.randint(low=0, high=len(all_imgs_path_list)-1, size=None, dtype=int)
Image.open(all_imgs_path_list[random_img_index])

In [6]:
#Importing the dataset and read the text file into a seperate variable

def load_doc(filename):
    
    text=open(filename).read()
    return text

doc = load_doc(captions_file_path)
print(doc[:500])

In [7]:
captions_df = pd.read_csv(captions_file_path) # Creating a captions dataframe with comma seperated values of images & captions 

print("Total captions present in the dataset: ", len(list(captions_df.caption)))
print("Total images present in the dataset: ",len(all_imgs_path_list))

captions_df.head()

## Each image id has 5 captions associated with it therefore the total dataset should have 8091x5 = 40455 samples.

In [8]:
all_img_id = list(captions_df.image)    # store all the image id here as list
all_img_vector = all_imgs_path_list     # store all the image path here as list
annotations = list(captions_df.caption) # store all the captions here as list

df = pd.DataFrame(list(zip(all_img_id, all_img_vector,annotations)),columns =['Image-ID','Path', 'Captions']) 
# df is dataframe containing image-id, image path and caption    
df.head(10)

In [9]:
# Adding the <start> & <end> token to all those captions as well

start_token = '<start>'
end_token = '<end>'

image_path_to_caption = defaultdict(list) # Dictionary with key: image id, values: list of 5 captions 

for idx, row in captions_df.iterrows():
    caption = f'{start_token} {row.caption} {end_token}' # adding <start> & <end> token for each caption
    image_path = os.path.join(images_folder_path,row.image) # image_path contains list of image paths
    image_path_to_caption[image_path].append(caption) # Generates dictionary of image_path: caption

In [10]:
# image_path_to_caption 

In [11]:
# Visualizing a random image and its captions

image_num = np.random.randint(low=0, high=len(all_imgs_path_list)-1, size=None, dtype=int)
print(list(image_path_to_caption.values())[image_num])
Image.open(list(image_path_to_caption.keys())[image_num])

In [12]:
# Visualizing image and corresponding captions

img_path_from_dict = list(image_path_to_caption.keys())
all_captions = []  # Caption list containing captions with start & end token for all the images
img_name_vector = [] # Creating image path list (ordered based on caption sets of 5, captions with start & end token)

for img_path in img_path_from_dict:
    
    # Caption list containing 5 captions with start & end token corresponding to a single image
    caption_list = image_path_to_caption[img_path] 
    
    # Caption list containing captions with start & end token for all images
    all_captions.extend(caption_list)
    
    # Creating image path list (ordered based on caption sets of 5, captions with start & end token)
    img_name_vector.extend([img_path] * len(caption_list))

# random image index 
image_num = np.random.randint(low=0, high=len(all_imgs_path_list)-1, size=None, dtype=int)
image_num = image_num//5
image_num = image_num*5

f, axes = plt.subplots(1, 2)
plt.axis('off')
f.set_figwidth(10)
axes[0].imshow(io.imread(img_name_vector[image_num]))
axes[0].axis('off')
axes[1].set_ylim(0,5)
for i in range(0,5):
    axes[1].text(0,i,all_captions[i+image_num])

In [13]:
# Creating the vocabulary & the counter for the captions

vocabulary= [word.lower() for line in annotations for word in line.split()]

val_count=Counter(vocabulary)

In [14]:
# val_count

In [15]:
# Visualising the top 30 occuring words in the captions

top30_words_countwise = val_count.most_common(30)
top30_words_df = pd.DataFrame(top30_words_countwise, columns = ['Word', 'Count'])
print(top30_words_df)

fig = plt.figure(figsize=(14,8))
sns.barplot(x='Word', y='Count', data = top30_words_df)
plt.title("Top 30 maximum frequency words", fontsize = 18)
plt.xlabel("Words", fontsize = 14)
plt.ylabel("Count", fontsize = 14)

<a id="section-4"></a>
# Pre-processing the captions
1.Creating the tokenized vectors by tokenizing the captions fore ex :split them using spaces & other filters. 
This gives a vocabulary of all of the unique words in the data. The total vocaublary is set to top 5,000 words for saving memory.

2.Replacing all other words with the unknown token "UNK" .

3.Creating word-to-index and index-to-word mappings.

4.Padding all sequences to be the same length as the longest one.

In [16]:
# creating the tokenizer

# Top 5000 words
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=5000, oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
# annotations =  list of all captions 
# fit_on_texts >> Updates internal vocabulary based on a list of texts, in case of texts containing lists, each entry of the lists is assumed to be a token.
# Required before using `texts_to_sequences`
tokenizer.fit_on_texts(all_captions) 
# Transforms each text in texts to a sequence of integers
tokenized_text_sequence = tokenizer.texts_to_sequences(all_captions)
tokenized_text_sequence[:5]

In [17]:
# Creating word-to-index and index-to-word mappings.

tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'
tokenized_text_sequence = tokenizer.texts_to_sequences(all_captions)
tokenized_text_sequence[:5]

In [18]:
random_index = np.random.randint(low=0, high=len(all_imgs_path_list)-1, size=None, dtype=int)
print(all_captions[random_index])
print(tokenized_text_sequence[random_index])

In [19]:
# Creating a word count of your tokenizer to visulize the Top 30 occuring words after text processing

dict_word_count = tokenizer.word_counts

In [20]:
# dict_word_count

In [21]:
# Visualising the top 30 occuring words in the captions after text processing

tokenized_word_count = sorted(tokenizer.word_counts.items(), key=lambda x:x[1], reverse=True)
tokenized_word_count_df = pd.DataFrame(tokenized_word_count, columns = ['Word', 'Count'])
top30_tokenized_words_df = tokenized_word_count_df.iloc[0:30,:]
print(top30_tokenized_words_df)

fig = plt.figure(figsize=(20,10))
sns.barplot(x='Word', y='Count', data = top30_tokenized_words_df)
plt.title("Top 30 maximum frequency words", fontsize = 18)
plt.xlabel("Words", fontsize = 14)
plt.ylabel("Count", fontsize = 14)

In [22]:
# all_captions - all captions list including start & end token
all_tokenized_text_sequence = tokenizer.texts_to_sequences(all_captions) 

max_length_caption = max(len(i) for i in all_tokenized_text_sequence)
print('Maximum length of any caption in the dataset: ', max_length_caption)

In [23]:
# Padding each vector to the max_length of the captions ^ store it to a vairable

cap_vector = tf.keras.preprocessing.sequence.pad_sequences(all_tokenized_text_sequence, padding = 'post',maxlen = max_length_caption)

print("The shape of Caption vector is :" + str(cap_vector.shape))
cap_vector[1:3,:]

<a id="section-5"></a>
# Pre-processing the images

* Resizing the image to the shape of (299, 299)
* Normalize the image within the range of -1 to 1, such that it is in correct format for InceptionV3.
#### Note on resizing the images:
* Since the list contains all the image path, we first convert them to a dataset using <i>tf.data.Dataset.from_tensor_slices</i>. Once the dataset is created, consisting of image paths, we apply a function to the dataset which will apply the necessary preprocessing to each image. 
* This function would resize them and also do the necessary preprocessing correct format of InceptionV3.

In [24]:
# Creating the function which returns images & their path

def load_image(image_path):
    #write your pre-processing steps here
    preprocessed_img = tf.io.read_file(image_path)
    preprocessed_img = tf.image.decode_jpeg(preprocessed_img, channels=3)
    preprocessed_img = tf.image.resize(preprocessed_img, (299, 299))
    # tf.keras.applications.inception_v3.preprocess_input will normalize input to range of -1 to 1
    preprocessed_img = tf.keras.applications.inception_v3.preprocess_input(preprocessed_img)
    return preprocessed_img, image_path

In [25]:
# Applying the function to the image path dataset, such that the transformed dataset would contain images & their path

encode_train_set = sorted(set(img_name_vector))    # img_name_vector - path list of all images according to caption set with start & end token

image_data_set = tf.data.Dataset.from_tensor_slices(encode_train_set)
image_data_set = image_data_set.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(32)
image_data_set


<a id="section-6"></a>
# Extracting the Feature vector
#### Loading the pretrained Imagenet weights of Inception net V3

1.To save the memory(RAM) from getting exhausted, we extract the features of the images using the last layer of pre-trained model. Including this as part of training will lead to higher computational time.

2.The shape of the output of this layer is 8x8x2048. 

3.Using a function to extract the features of each image in the train & test dataset such that the shape of each image would be (batch_size, 8*8, 2048)
##### Note on storing the features:
* We can store the features using a dictionary with the path as the key and the feature extracted by the inception net v3 model as values. OR
* We can store using numpy(np.save) to store the resulting vector.

In [26]:
image_model = tf.keras.applications.InceptionV3(include_top=False,weights='imagenet')

new_input = image_model.input                    #write code here to get the input of the image_model
hidden_layer = image_model.layers[-1].output     #write code here to get the output of the image_model

image_features_extract_model = keras.Model(new_input, hidden_layer)   #build the final model using both input & output layer

In [27]:
image_features_extract_model.summary()

In [28]:
# Applying the feature_extraction model to earlier created dataset which contained images & their respective paths

feature_dict = {}

# using tqdm for progress bar
for image,path in tqdm(image_data_set):  
    
    # feed images from newly created Dataset above to Inception V3 built above    
    batch_features = image_features_extract_model(image)
    
    # To squeeze out the features in a batch, to reshape features in order of (batch_size, 8*8, 2048)    
    batch_features_reshaped = tf.reshape(batch_features, (batch_features.shape[0], -1, batch_features.shape[3]))
    
    for batch_f, p in zip(batch_features_reshaped, path):
        path_of_feature = p.numpy().decode("utf-8")
        feature_dict[path_of_feature] =  batch_f.numpy()

In [29]:
image_data_set

In [30]:
# batch_features

In [31]:
# batch_features_reshaped

In [32]:
print('Shape of batch_features - ',batch_features.shape)
print('Shape of batch_features_reshaped - ',batch_features_reshaped.shape)
print('Shape of feature extracted for an image',batch_f.shape)

<a id="section-7"></a>
# Model Building

<a id="subsect-1"></a>
# Data preperation
1. Applying train_test_split on both image path & captions to create the train & test list. Creating the train-test spliit using 80-20 ratio & random state = 42
2. Creating a function which maps the image path to their feature. 
3. Creating a builder function to create train & test dataset & apply the function created earlier to transform the dataset
4. Shuffle the batch while building the dataset
5. The shape of each image in the dataset after building would be (batch_size, 8*8, 2048)
6. The shape of each caption in the dataset after building would be(batch_size, max_len)

#### Note on loading the features:
* We can load the features using the dictionary created earlier. OR
* We can store using numpy(np.load) to load the feature vector.

In [33]:
# Applying train_test_split on both image path & captions to create the train & test list. 
# Creating the train-test spliit using 80-20 ratio & random state = 42

path_train, path_test, cap_train, cap_test = train_test_split(img_name_vector, cap_vector, test_size=0.2, random_state=42)
print("Training data for images: " + str(len(path_train)))
print("Testing data for images: " + str(len(path_test)))
print("Training data for Captions: " + str(len(cap_train)))
print("Testing data for Captions: " + str(len(cap_test)))

In [34]:
# Creating a function which maps the image path to their feature. 
# This function will take the image_path & caption and return it's feature & respective caption.

def map_func(image_name,capt):     
  image_tensor = feature_dict[image_name.decode('utf-8')]    # Code to extract the features from the dictionary stored earlier
  return image_tensor,capt

In [35]:
# creating a builder function to create dataset which takes in the image path & captions as input
# This function would transform the created dataset(img_path,cap) to (features,cap) using the map_func created earlier

BATCH_SIZE = 32
BUFFER_SIZE = 1000
def gen_dataset(images_data, captions_data):
        
    dataset = tf.data.Dataset.from_tensor_slices((images_data, captions_data))
    dataset = dataset.shuffle(BUFFER_SIZE)
 
    dataset = dataset.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(BATCH_SIZE)

    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

In [36]:
train_dataset=gen_dataset(path_train,cap_train)
test_dataset=gen_dataset(path_test,cap_test)

In [37]:
sample_img_batch, sample_cap_batch = next(iter(train_dataset))
print(sample_img_batch.shape)  #(batch_size, 8*8, 2048)
print(sample_cap_batch.shape) #(batch_size,max_len)

In [38]:
# Setting the parameters

embedding_dim = 256 
units = 512
vocab_size = 5001   #top 5,000 words +1
train_num_steps = len(path_train) // BATCH_SIZE
test_num_steps = len(path_test) // BATCH_SIZE

features_shape = batch_f.shape[1]
attention_features_shape = batch_f.shape[0]

<a id="subsect-2"></a>
# Building the Encoder

In [39]:
class Encoder(Model):
    def __init__(self,embedding_dim):
        super(Encoder, self).__init__()
        self.dense = layers.Dense(embedding_dim)       #build your Dense layer with relu activation
        
    def call(self, features, training=False):
        # extract the features from the image shape: (batch, 8*8, embed_dim)
        features = self.dense(features)
        features = tf.nn.relu(features)
        return features

In [40]:
encoder=Encoder(embedding_dim)

<a id="subsect-3"></a>
# Building the Attention model

In [41]:
class Attention_model(Model):
    def __init__(self, units):
        super(Attention_model, self).__init__()
        self.W1 = layers.Dense(units)       #build Dense layer
        self.W2 = layers.Dense(units)       #build Dense layer
        self.V = layers.Dense(1)            #build final Dense layer with unit 1
        self.units=units

    def call(self, features, hidden):
        #features shape: (batch_size, 8*8, embedding_dim)
        # hidden shape: (batch_size, hidden_size)
        
        # Expand the hidden shape to shape: (batch_size, 1, hidden_size)
        hidden_with_time_axis =  tf.expand_dims(hidden, 1)  
        
        # build the score funciton to shape: (batch_size, 8*8, units)
        score = keras.activations.tanh(self.W1(features) + self.W2(hidden_with_time_axis)) 
        
        # extract the attention weights with shape: (batch_size, 8*8, 1)
        attention_weights = keras.activations.softmax(self.V(score), axis=1)
        
        #shape: create the context vector with shape (batch_size, 8*8,embedding_dim)
        context_vector = attention_weights * features 
        
        # reduce the shape to (batch_size, embedding_dim)
        context_vector = tf.reduce_sum(context_vector, axis=1)
    
        return context_vector, attention_weights

<a id="subsect-4"></a>
# Building the Decoder

In [42]:
class Decoder(Model):
    def __init__(self, embed_dim, units, vocab_size):
        super(Decoder, self).__init__()
        self.units=units
        self.attention = Attention_model(self.units)                # iniitalise the Attention model with units
        self.embed = layers.Embedding(vocab_size, embedding_dim)    # build the Embedding layer     
        self.gru = layers.GRU(self.units,return_sequences=True,return_state=True,recurrent_initializer='glorot_uniform')
        self.d1 = layers.Dense(self.units)              # build the Dense layer    
        self.d2 = layers.Dense(vocab_size)              # build the Dense layer
        
    def call(self,x,features, hidden):
        #create the context vector & attention weights from attention model
        context_vector, attention_weights = self.attention(features, hidden)
        # embed the input to shape: (batch_size, 1, embedding_dim)
        embed = self.embed(x) 
        # Concatenate the input with the context vector from attention layer. Shape: (batch_size, 1, embedding_dim + embedding_dim)
        embed = tf.concat([tf.expand_dims(context_vector, 1), embed], axis=-1) 
        # Extract the output & hidden state from GRU layer. Output shape : (batch_size, max_length, hidden_size)
        output,state = self.gru(embed)
        output = self.d1(output)
        # shape : (batch_size * max_length, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2])) 
        # shape : (batch_size * max_length, vocab_size)
        output = self.d2(output) 
        
        return output, state, attention_weights
    
    def init_state(self, batch_size):
        return tf.zeros((batch_size, self.units))

In [43]:
decoder=Decoder(embedding_dim, units, vocab_size)

In [44]:
features=encoder(sample_img_batch)

hidden = decoder.init_state(batch_size=sample_cap_batch.shape[0])
dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * sample_cap_batch.shape[0], 1)

predictions, hidden_out, attention_weights= decoder(dec_input, features, hidden)
print('Feature shape from Encoder: {}'.format(features.shape)) #(batch, 8*8, embed_dim)
print('Predcitions shape from Decoder: {}'.format(predictions.shape)) #(batch,vocab_size)
print('Attention weights shape from Decoder: {}'.format(attention_weights.shape)) #(batch, 8*8, embed_dim)

<a id="section-8"></a>
# Model training & optimization

1. Set the optimizer & loss object
2. Create the checkpoint path
3. Create the training & testing step functions
4. Create the loss function for the test dataset

In [45]:
LR = 2e-4
optimizer = tf.keras.optimizers.Adam(learning_rate = LR)         #define the optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True, reduction='none')   #define the loss object

In [46]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [47]:
checkpoint_path = os.path.join(working_folder_path,'checkpoint')
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [48]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])

### * While creating the training step for our model, we apply Teacher forcing.
#### * Teacher forcing is a technique where the target/real word is passed as the next input to the decoder instead of previous prediciton.

In [49]:
@tf.function
def train_step(img_tensor, target):
    loss = 0
    hidden = decoder.init_state(batch_size=target.shape[0])
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)
    
    with tf.GradientTape() as tape:
        features = encoder(img_tensor)
        for i in range(1, target.shape[1]):
            predictions, hidden, _ = decoder(dec_input, features, hidden)
            loss += loss_function(target[:, i], predictions)
            dec_input = tf.expand_dims(target[:, i], 1)
            
    avg_loss = (loss/int(target.shape[1]))
    
    trainable_variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))     
                       
    return loss, avg_loss

### * While creating the test step for our model, we pass oour previous prediciton as the next input to the decoder.

In [50]:
@tf.function
def test_step(img_tensor, target):
    loss = 0
    hidden = decoder.init_state(batch_size=target.shape[0])
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)

    with tf.GradientTape() as tape:
        features = encoder(img_tensor)
        for i in range(1, target.shape[1]):
            predictions, hidden, _ = decoder(dec_input, features, hidden)
            loss += loss_function(target[:, i], predictions)
            dec_input = tf.expand_dims(target[:, i], 1)

    avg_loss = (loss / int(target.shape[1]))

    trainable_variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))

    
    return loss, avg_loss

In [51]:
def test_loss_cal(test_dataset):
    total_loss = 0
    
    # Code to get the average loss result on our test data
    for (batch, (img_tensor, target)) in enumerate(test_dataset):
        batch_loss, t_loss = test_step(img_tensor, target)
        total_loss += t_loss
    avg_test_loss=total_loss/test_num_steps
    
    return avg_test_loss

In [52]:
loss_plot = []
test_loss_plot = []
EPOCHS = 15

best_test_loss=100
for epoch in tqdm(range(0, EPOCHS)):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(train_dataset):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss
        avg_train_loss=total_loss / train_num_steps
        
    loss_plot.append(avg_train_loss)    
    test_loss = test_loss_cal(test_dataset)
    test_loss_plot.append(test_loss)
    
    print ('For epoch: {}, the train loss is {:.3f}, & test loss is {:.3f}'.format(epoch+1,avg_train_loss,test_loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
    
    if test_loss < best_test_loss:
        print('Test loss has been reduced from %.3f to %.3f' % (best_test_loss, test_loss))
        best_test_loss = test_loss
        ckpt_manager.save()

### Final Test loss after 15 epochs = 0.611

In [53]:
plt.figure(figsize=(10, 7))
plt.plot(loss_plot)
plt.plot(test_loss_plot)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training & Test loss plot wrt number of epochs')
plt.legend()
plt.show()

#### NOTE: 
* Since there is a difference between the train & test steps ( Presence of teacher forcing), we may observe that the train loss is decreasing while your test loss is not. 
* This doesn't mean that the model is overfitting, as we can't compare the train & test results here, as both approach is different.
* Also, if we want to achieve better results we can run it more epochs, but the intent of this project is to get an idea on how to integrate attention mechanism with E-D architecture for images. The intent is not to create the state of art model. 

<a id="section-9"></a>
# Model Evaluation

1. Define the evaluation function using greedy search
2. Define the evaluation function using beam search
3. Test it on a sample data using BLEU score

### Greedy Search

In [54]:
def evaluate(image):
    attention_plot = np.zeros((max_length_caption, attention_features_shape))

    hidden = decoder.init_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)  # process the input image to desired format before extracting features
    img_tensor_val = image_features_extract_model(temp_input)        # Extracting features using feature extraction model
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)       # extracting the features by passing the input to encoder

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length_caption):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)    # getting the output from decoder

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()
        
        # extract the predicted id(embedded value) which carries the max value
        predicted_id = tf.argmax(predictions[0]).numpy()     
        # map the id to the word from tokenizer and append the value to the result list
        result.append(tokenizer.index_word[predicted_id])
        
        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot,predictions

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot,predictions


### Beam Search

In [55]:
def beam_evaluate(image, beam_index = 3):#your value for beam index):
    
    # to evaluate the result using beam search
    start = [tokenizer.word_index['<start>']]
    result = [[start, 0.0]]

    attention_plot = np.zeros((max_length_caption, attention_features_shape))

    hidden = decoder.init_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)

    while len(result[0][0]) < max_length_caption:
        i=0
        temp = []
        for s in result:
            predictions, hidden, attention_weights = decoder(dec_input, features, hidden)
            attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()
            i=i+1
            word_preds = np.argsort(predictions[0])[-beam_index:]
          
            for w in word_preds:
                next_cap, prob = s[0][:], s[1]
                next_cap.append(w)
            
                prob += np.log(predictions[0][w])
                    
                temp.append([next_cap, prob])
        result = temp
        result = sorted(result, reverse=False, key=lambda l: l[1])
        result = result[-beam_index:]
        
        
        predicted_id = result[-1]
        pred_list = predicted_id[0]
        
        prd_id = pred_list[-1] 
        if(prd_id!=3):
            dec_input = tf.expand_dims([prd_id], 0)  
        else:
            break
    
    
    result2 = result[-1][0]
    
    intermediate_caption = [tokenizer.index_word[i] for i in result2]
    final_caption = []
    for i in intermediate_caption:
        if i != '<end>':
            final_caption.append(i)
            
        else:
            break

    attention_plot = attention_plot[:len(result), :]
    final_caption = ' '.join(final_caption[1:])
    
    return final_caption



In [56]:
def plot_attention_map(caption, weights, image):

    fig = plt.figure(figsize=(10, 10))
    temp_img = np.array(Image.open(image))
    
    len_cap = len(caption)
    for cap in range(len_cap):
        weights_img = np.reshape(weights[cap], (8,8))
        weights_img = np.array(Image.fromarray(weights_img).resize((224, 224), Image.LANCZOS))
        
        ax = fig.add_subplot(len_cap//2, len_cap//2, cap+1)
        ax.set_title(caption[cap], fontsize=15)
        
        img=ax.imshow(temp_img)
        
        ax.imshow(weights_img, cmap='gist_heat', alpha=0.6,extent=img.get_extent())
        ax.axis('off')
    plt.subplots_adjust(hspace=0.2, wspace=0.2)
    plt.show()

In [57]:
def filt_text(text):
    filt=['<start>','<unk>','<end>'] 
    temp= text.split()
    [temp.remove(j) for k in filt for j in temp if k==j]
    text=' '.join(temp)
    return text

In [58]:
def caption_prediction(autoplay=False, weights=(0.5, 0.5, 0, 0)):
    rid = np.random.randint(0, len(path_test))
    test_image = path_test[rid]
    #test_image = './images/413231421_43833a11f5.jpg'
    #real_caption = '<start> black dog is digging in the snow <end>'

    real_caption = ' '.join([tokenizer.index_word[i] for i in cap_test[rid] if i not in [0]])
    result, attention_plot, pred_test = evaluate(test_image)
    

    real_caption=filt_text(real_caption)      

    
    pred_caption=' '.join(result).rsplit(' ', 1)[0]

    real_appn = []
    real_appn.append(real_caption.split())
    reference = real_appn
    candidate = pred_caption.split()
    
    score = sentence_bleu(reference, candidate, weights = weights)     #set your weights)
    print(f"BELU score: {score*100}")
    
    print('Real Caption:', real_caption)
    print(f"Prediction Caption: {pred_caption}")
    plot_attention_map(result, attention_plot, test_image)
    
    # we make use of Google Text to Speech API (online), which will convert the caption to audio
    speech = gTTS('Predicted Caption : ' + pred_caption, lang = 'en', slow = False)
    speech.save('voice.mp3')
    audio_file = 'voice.mp3'

    display.display(display.Audio(audio_file, rate = None, autoplay = autoplay))
    
    return Image.open(test_image)

#### Testing image - 1

In [59]:
caption_prediction(autoplay=True, weights=(0.5, 0.5, 0, 0))

#### Testing image - 2

In [60]:
caption_prediction(autoplay=True, weights=(0.25, 0.25, 0, 0))

#### Testing image - 3

In [61]:
caption_prediction(autoplay=True, weights=(0.2, 0.3, 0.4, 0.5))

#### Testing image - 4

In [63]:
caption_prediction(autoplay=True, weights=(0.5, 0.4, 0.3, 0.2))

#### Testing image - 5

In [64]:
caption_prediction(autoplay=True, weights=(0.25, 0.25, 0.25, 0.25))

#### Testing image - 6

In [65]:
caption_prediction(autoplay=True, weights=(0.3, 0.3, 0.4, 0.4))

<a id="section-10"></a>
# Conclusions

#### * Inception V3 Model with pretrained weights (Imagenet)was used to extract feature vectors
#### * Attention model is used to extract context vectors before feeding into decoder/RNN
#### * Testing was carried out with different combinations of weights